In [ ]:
import numpy as np
import random
import string
rnd=np.random
rnd.seed(0)

In [ ]:
# nber of items
nb_items=10
list_article=[]
for index in range(1,nb_items+10):
    item=random.choice(string.ascii_lowercase)
    while item not in list_article and len(list_article)<10:
        list_article.append(item)
print(list_article)

# nber of clients
nb_clients=5
N=[iorder for iorder in range(1,nb_clients+1)]

# nber of pickers
nber_pickers=4
Batch= [jbatch for jbatch in range(1,nber_pickers+1)]
Q=100                 # Batch capacity

# nber of aisle
K=3     
G=[kaisle for kaisle in range(1,K+1)]
L=100
W=40

In [ ]:
#Orders and quantities
Qte=range(1,20)
for iorder in N:
    limit=rnd.randint(1,9)
    Order ={iorder:random.sample(list_article,limit)}
    print(Order)
   # q={iorder:random.sample(Qte, limit)}
    qte=random.sample(Qte, limit)
    print(qte)
    sum=0
    for elem in range(0,len(qte)):
        sum=sum+qte[elem]
        mqte=sum
    print(mqte)

In [ ]:
#articles's location
val_x=[]
loc_x=[]
loc_y=[]
xabs=[0,40,80]
for kaisle in G: 
    x_coord= { kaisle:xabs[kaisle-1]}
    val_x.append(x_coord[kaisle])
print(val_x)
for elem in range(0,nb_items):
    loc_x.append(random.choice(val_x))
    
loc_y= rnd.rand(10)*100
loc_x.append(0)
loc_y=np.append(loc_y,0)
print(loc_x)
print(loc_y)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Layout
plt.scatter(loc_x[0:10],loc_y[0:10],c='b')
for elem in range(0,nb_items):
    plt.annotate( "%s" %(list_article[elem]),(loc_x[elem]+2,loc_y[elem]))
plt.plot(loc_x[10],loc_y[10], c='r', marker='s')
plt.axis('equal');

In [ ]:
# set
A=[(iorder,jbatch) for iorder in N for jbatch in Batch]
B=[(jbatch, kaisle) for jbatch in Batch for  kaisle in G ]
C=[(iorder, kaisle) for iorder in N for  kaisle in G ]

In [ ]:
from gurobipy import*

In [ ]:
mdl = Model("OBP")

In [ ]:
xvar=mdl.addVars(A,vtype=GRB.BINARY,name='xvar')
pvar=mdl.addVars(B, vtype=GRB.BINARY,name='pvar')
yvar=mdl.addVars(B, vtype=GRB.BINARY,name='yvar')
cvar=mdl.addVars(Batch, vtype=GRB.BINARY, name='cvar')
h_rvar=mdl.addVars(Batch,vtype=GRB.CONTINUOUS, name='h_rvar')
h_lvar=mdl.addVars(Batch,vtype=GRB.CONTINUOUS, name='h_lvar')
uvar=mdl.addVars(B,vtype=GRB.CONTINUOUS,name='uvar')
vvar=mdl.addVars(Batch,vtype=GRB.INTEGER, name='vvar')

In [ ]:
#Model
mdl.setObjective(
    2*quicksum(quicksum(uvar[jbatch,kaisle] for jbatch in Batch for kaisle in G))+ 
    2*quicksum( h_rvar[jbatch]+h_lvar[jbatch] for jbatch in Batch)+ 
    2*L*quicksum(vvar[jbatch]-cvar[jbatch] for jbatch in Batch), GRB.MINIMIZE)

mdl.addConstr(quicksum(xvar[iorder,jbatch] for jbatch in Batch)==1 for iorder in N)
mdl.addConstr(xvar[iorder,jbatch]<=xvar[jbatch,jbatch] for iorder in N for jbatch in Batch)
mdl.addConstr(quicksum(mqte[iorder]*xvar[iorder,jbatch] for iorder in N)<=Q for jbatch in Batch)
mdl.addConstr(yvar[jbatch,kaisle]<= quicksum(dvar[iorder,kaisle]*xvar[iorder,jbatch] for iorder in N)<=M*yvar[jbatch,kaisle] for jbatch in Batch for kaisle in G)
mdl.addConstr((kaisle-ka)*W*yvar[jbatch,kaisle]<= h_rvar[jbatch] for jbatch in Batch for kaisle in range(ka+1,K))
mdl.addConstr((ka-kaisle)*W*yvar[jbatch,kaisle]<= h_lvar[jbatch] for jbatch in Batch for kaisle in range(1,ka-1))
mdl.addConstr(quicksum(yvar[jbatch,kaisle] for kaisle in G)+cvar[jbatch]=2*vvar[jbatch] for jbatch in Batch)
mdl.addConstr(dvar[iorder,kaisle]*xvar[iorder,jbatch]<= uvar[jbatch,kaisle]+M(1-pvar[jbatch,kaisle])+M(1-cvar[jbatch]) for iorder in N for jbatch in Batch for kaisle in range(2,K))
mdl.addConstr(yvar[jbatch,kaisle]- quicksum(yvar[jbatch,lvar] for lvar in range(kaisle+1,K))<= pvar[jbatch,kaisle]<=yvar[jbatch,kaisle] for jbatch in Batch for kaisle in G)


model.optimize()

In [ ]:
#solution
vals = mdl.getAttr('xvar', xvar)
selected = gp.tuplelist((iorder,jbatch) for iorder,jbatch in vals.keys() if vals[iorder,jbatch] > 0.5)
print(selected)
print ("Optimal value=", int(model.ObjVal))
print("Time elapsed:", time.perf_counter() -start, "s")